In [1]:
%load_ext tensorboard
%load_ext dotenv

%dotenv

In [2]:
import os
import openai
from pydantic import BaseModel

MODEL = "gpt-4o-mini"

openai.api_key = os.getenv("OPENAI_API_KEY")

In [71]:
x="Hospital Management System"

In [72]:
from typing import Literal

from pydantic import Field

# note: maybe a different prompt would work better, i.e. name people that could be involved with the topic, then ask to generate them, getting a more diverse list potentially?

generate_personas_prompt = """You are an ontology engineer tasked with creating an ontology on <topic>{topic}</topic>. Find {count} individuals who you would like to interview to figure out initial competency questions. Ensure a diverse mix of backgrounds, experiences, and viewpoints. Include people from different professions, levels of education, socioeconomic statuses, geographical locations, and lived experiences. Focus on real-world perspectives from all walks of life. The goal is to avoid over-representing a single demographic and instead capture a broad spectrum of voices that could contribute valuable insights on this topic."""


class Education(BaseModel):
    institution: str
    degree: str | None
    field_of_study: str | None
    description: str | None

class WorkExperience(BaseModel):
    company: str
    position: str
    description: str | None

class Skill(BaseModel):
    name: str
    proficiency: Literal["beginner", "intermediate", "advanced"] | None

class Persona(BaseModel):
    name: str
    bio: str = Field(description="Short bio written by the person themselves. (\"I am ...\")")
    education: list[Education] | None
    work_experience: list[WorkExperience] | None
    skills: list[Skill] | None



class Personas(BaseModel):
    items: list[Persona]

def generate_personas(topic: str, count: int):
    prompt = generate_personas_prompt.format(count=count, topic=topic)

    response = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt}
        ],
        response_format=Personas
    )

    # todo make sure we have the right number of personas, else generate more

    return response.choices[0].message.parsed

personas = generate_personas(x, 10)

print(personas.model_dump_json(indent=True))

{
 "items": [
  {
   "name": "Dr. Emily Chen",
   "bio": "I am a hospital administrator with over 15 years of experience managing healthcare facilities.",
   "education": [
    {
     "institution": "Harvard University",
     "degree": "Ph.D.",
     "field_of_study": "Healthcare Management",
     "description": "Research-focused on improving patient care through management practices."
    },
    {
     "institution": "Johns Hopkins University",
     "degree": "M.P.H.",
     "field_of_study": "Public Health",
     "description": "Focused on health policy and hospital administration."
    }
   ],
   "work_experience": [
    {
     "company": "City General Hospital",
     "position": "Chief Administrator",
     "description": "Oversee the daily operational management and strategic planning."
    },
    {
     "company": "Metropolitan Health System",
     "position": "Operations Manager",
     "description": "Implemented policies to improve patient flow and reduce wait times."
    }
   ],


In [73]:
from typing import Any


expert_system_prompt = "You are {description}. You are being interviewed by an ontology engineer who is tasked with creating an ontology on <topic>{topic}</topic>. DO NOT repeat information that is already clear, but feel free to ask for clarification if needed. Keep your answers concise and to the point. This should be a conversation!"

engineer_system_prompt = "You are an ontology engineer tasked with creating an ontology on <topic>{topic}</topic>. You are interviewing {description}. Your goal is to understand the domain from the expert's point of view, to define the competency questions that will guide the creation of the ontology and to clear up any ambiguities you have. DO NOT repeat information that is already clear, but feel free to ask for clarification if needed. However, note that the expert may NOT know about ontologies and competency questions, defining these is your job. Keep your answers concise and to the point. This should be a conversation!"

Speaker = Literal["expert", "engineer"]

class Turn(BaseModel):
    """A turn in the conversation between the expert and the engineer"""

    role: Speaker
    content: str

    def to_openai_format(self, speaker: Speaker) -> Any:
        return {"role": ("assistant" if self.role == speaker else "user"), "content": self.content}

class Interview(BaseModel):
    persona: Persona
    topic: str
    protocol: list[Turn]

    def add_turn(self, role: Speaker, content: str):
        self.protocol.append(Turn(role=role, content=content))


class CompetencyQuestion(BaseModel):
    question: str
    confidence: float

    notes: str | None = Field(description="Any notes or context that might be relevant to this question, especially if you are not sure this works. Maybe also just ask the expert!")




class DefineCompetencyQuestionAction(BaseModel):
    """Define a competency question"""
    
    type: Literal["def_cq"]
    cq: CompetencyQuestion

class Thought(BaseModel):
    chain_of_thought: str = Field(description="Your INTERNAL MONOLOGUE. Think about what was just said and what you should do next!")

    actions: list[DefineCompetencyQuestionAction]

    response: str = Field(description="Your response to the expert. Keep it as concise as possible.")

class Memory(BaseModel):
    questions: list[CompetencyQuestion]
    thoughts: list[Thought]




def expert_respond(interview: Interview):
    """Simulate the expert's response to the engineer's """

    
    sys_prompt = expert_system_prompt.format(description=interview.persona.model_dump_json(), topic=interview.topic)


    response = openai.chat.completions.create(
            model=MODEL,
            messages=[{"role": "system", "content": sys_prompt},
                      *[turn.to_openai_format("expert") for turn in interview.protocol]
            ]
        )
    
    return response.choices[0].message.content

def engineer_think(interview: Interview):
    """Simulate the engineer's thought process after the expert's response"""

    sys_prompt = engineer_system_prompt.format(description="You are an ontology engineer ", topic=interview.topic)

    response = openai.beta.chat.completions.parse(
            model=MODEL,
            messages=[{"role": "system", "content": sys_prompt},
                      *[turn.to_openai_format("engineer") for turn in interview.protocol]
            ],
            response_format=Thought
        )

    return response.choices[0].message.parsed

def interview(persona: Persona, topic: str):
    """A simulated interview between a domain expert and an ontology engineer"""

    interview = Interview(persona=persona, topic=topic, protocol=[
        Turn(role="engineer", content="Thank you for taking the time to speak with me. My goal today is to better understand your domain and how knowledge is structured within it. There's no fixed script, so I'd love to hear your thoughts and experiences naturally. Can you describe your field and what kind of problems you typically deal with?"),
    ])

    memory = Memory(questions=[], thoughts=[])

    while True:
        response = expert_respond(interview)

        if not response:
            raise Exception("No response from expert, something weird happened")

        interview.add_turn("expert", response)

        print(f"{persona.name}: {response}")

        thought = engineer_think(interview)

        if not thought:
            raise Exception("No response from engineer, something weird happened")
        
        interview.add_turn("engineer", thought.response)
        
        memory.thoughts.append(thought)
        # append new cqs
        for action in thought.actions:
            if isinstance(action, DefineCompetencyQuestionAction):
                memory.questions.append(action.cq)

        print(thought.model_dump_json(indent=True))

        print(f"Engineer: {thought.response}")


interview(personas.items[1], x)




Lisa Patel: Thank you for having me! In my field of public health and community advocacy, I focus on improving health equity, particularly for underserved populations. I typically deal with issues like healthcare access, health literacy, and disparities in health outcomes. This includes addressing barriers that vulnerable communities face when navigating the healthcare system, organizing outreach programs, and ensuring that these populations receive the necessary resources and information to improve their health.
{
 "chain_of_thought": "The expert is focused on public health and community advocacy, particularly related to health equity and access for underserved populations. This gives me insight into the kinds of entities and relationships that may be important for the ontology. I should explore how the hospital management system interacts with these community health initiatives and what specific aspects need to be represented in the ontology. I'll ask about key concepts in hospital m

KeyboardInterrupt: 